In [1]:
import findspark
findspark.init()

In [2]:
from pyspark.sql import SparkSession
spark = SparkSession.builder \
.master("local[4]") \
.appName("Null") \
.config("spark.driver.memory","4g") \
.config("spark.executor.memory","4g") \
.getOrCreate()

22/11/14 02:03:27 WARN Utils: Your hostname, batuhan-VirtualBox resolves to a loopback address: 127.0.1.1; using 10.0.2.15 instead (on interface enp0s3)
22/11/14 02:03:27 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


22/11/14 02:03:29 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
22/11/14 02:03:33 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [3]:
nullDF = spark.createDataFrame([
    (1,"Batu","Bursa",30003),
    (2,"Mehmet","İstanbul",30001),
    (3,"Ayşe","İzmir",None),
    (4,"Betül",None,3001),
    (5,"Şeyda","Tokat",30006),
    (None,None,None,None)],
    ["ID","Name","city","pin"]
                              )

In [4]:
nullDF.show()

+----+------+--------+-----+
|  ID|  Name|    city|  pin|
+----+------+--------+-----+
|   1|  Batu|   Bursa|30003|
|   2|Mehmet|İstanbul|30001|
|   3|  Ayşe|   İzmir| null|
|   4| Betül|    null| 3001|
|   5| Şeyda|   Tokat|30006|
|null|  null|    null| null|
+----+------+--------+-----+



In [5]:
nulldropDF = nullDF.dropna("any") #any or all
nulldropDF.show() # Any'nin anlamı null bulduğun herhangi bir colu sil

+---+------+--------+-----+
| ID|  Name|    city|  pin|
+---+------+--------+-----+
|  1|  Batu|   Bursa|30003|
|  2|Mehmet|İstanbul|30001|
|  5| Şeyda|   Tokat|30006|
+---+------+--------+-----+



In [6]:
nulldropDF2 = nullDF.dropna("all")
nulldropDF2.show() # all'da ise sadece tüm değerler null olduğunda siler yani ıd name city ve pin null olursa siler

+---+------+--------+-----+
| ID|  Name|    city|  pin|
+---+------+--------+-----+
|  1|  Batu|   Bursa|30003|
|  2|Mehmet|İstanbul|30001|
|  3|  Ayşe|   İzmir| null|
|  4| Betül|    null| 3001|
|  5| Şeyda|   Tokat|30006|
+---+------+--------+-----+



In [7]:
nulldropNameDF3 = nullDF.dropna(subset="Name") #burda ise sadece Name'de eğer null değerler var ise siler
#yni ıd city pin dolu olsun sadece name boş olsun siler fakat name dolu city boş olsa silmez
nulldropNameDF3.show()

+---+------+--------+-----+
| ID|  Name|    city|  pin|
+---+------+--------+-----+
|  1|  Batu|   Bursa|30003|
|  2|Mehmet|İstanbul|30001|
|  3|  Ayşe|   İzmir| null|
|  4| Betül|    null| 3001|
|  5| Şeyda|   Tokat|30006|
+---+------+--------+-----+



In [8]:
nulldropNameDF4 = nullDF.dropna(subset=("Name","pin")) # eğer name ve pinde boşluk varsa onları siler
nulldropNameDF4.show()

+---+------+--------+-----+
| ID|  Name|    city|  pin|
+---+------+--------+-----+
|  1|  Batu|   Bursa|30003|
|  2|Mehmet|İstanbul|30001|
|  4| Betül|    null| 3001|
|  5| Şeyda|   Tokat|30006|
+---+------+--------+-----+



In [9]:
nulldropNameDF5 = nullDF.dropna(thresh=2,subset=("ID","Name","city","pin"))
nulldropNameDF5.show()

+---+------+--------+-----+
| ID|  Name|    city|  pin|
+---+------+--------+-----+
|  1|  Batu|   Bursa|30003|
|  2|Mehmet|İstanbul|30001|
|  3|  Ayşe|   İzmir| null|
|  4| Betül|    null| 3001|
|  5| Şeyda|   Tokat|30006|
+---+------+--------+-----+



In [10]:
nulldropNameDF5 = nullDF.dropna(thresh=3,subset=("ID","Name","city","pin"))
nulldropNameDF5.show()

+---+------+--------+-----+
| ID|  Name|    city|  pin|
+---+------+--------+-----+
|  1|  Batu|   Bursa|30003|
|  2|Mehmet|İstanbul|30001|
|  3|  Ayşe|   İzmir| null|
|  4| Betül|    null| 3001|
|  5| Şeyda|   Tokat|30006|
+---+------+--------+-----+



# Thresh'in mantığı şu; en az kaç değerin boş olması gerektiğini söylüyor yani thresh 2 dersen 1 tane null
# olanı silmez 3 dersen 2 tane null olanı silmez

In [11]:
nullfillNameDF = nullDF.na.fill({"Name":"Batu"}) # pyspark name'deki her boş değeri kontrol ediyor ve boş olan
# name'lere batu yazıyor 
nullfillNameDF.show()

+----+------+--------+-----+
|  ID|  Name|    city|  pin|
+----+------+--------+-----+
|   1|  Batu|   Bursa|30003|
|   2|Mehmet|İstanbul|30001|
|   3|  Ayşe|   İzmir| null|
|   4| Betül|    null| 3001|
|   5| Şeyda|   Tokat|30006|
|null|  Batu|    null| null|
+----+------+--------+-----+



In [12]:
nullDF.filter(nullDF.city.isNull()).show()

+----+-----+----+----+
|  ID| Name|city| pin|
+----+-----+----+----+
|   4|Betül|null|3001|
|null| null|null|null|
+----+-----+----+----+



In [13]:
nullDF.filter(nullDF.pin.isNull()).show()

+----+----+-----+----+
|  ID|Name| city| pin|
+----+----+-----+----+
|   3|Ayşe|İzmir|null|
|null|null| null|null|
+----+----+-----+----+



In [14]:
nullDF.filter(nullDF.Name.isNull()).show()

+----+----+----+----+
|  ID|Name|city| pin|
+----+----+----+----+
|null|null|null|null|
+----+----+----+----+



In [15]:
nullDF.filter(nullDF.pin.isNotNull()).show()

+---+------+--------+-----+
| ID|  Name|    city|  pin|
+---+------+--------+-----+
|  1|  Batu|   Bursa|30003|
|  2|Mehmet|İstanbul|30001|
|  4| Betül|    null| 3001|
|  5| Şeyda|   Tokat|30006|
+---+------+--------+-----+



In [16]:
data_student = [("Joseph","Science",86,"p",90),
                ("Martin","Mathematics",90,"p",None),
                ("David","English",20,"F",80),
                ("John","Science",None,"F",None),
                ("Diana",None,30,"F",50),
                ("Ania","Mathematics",None,None,70)]

Schema=["name","Subject","Mark","Status","Attedance"]
df = spark.createDataFrame(data = data_student, schema = Schema)
df.show()

+------+-----------+----+------+---------+
|  name|    Subject|Mark|Status|Attedance|
+------+-----------+----+------+---------+
|Joseph|    Science|  86|     p|       90|
|Martin|Mathematics|  90|     p|     null|
| David|    English|  20|     F|       80|
|  John|    Science|null|     F|     null|
| Diana|       null|  30|     F|       50|
|  Ania|Mathematics|null|  null|       70|
+------+-----------+----+------+---------+



In [17]:
from pyspark.sql.functions import col

In [18]:
df2 = (df.filter(col("Mark").isNull()))
df2.show()

+----+-----------+----+------+---------+
|name|    Subject|Mark|Status|Attedance|
+----+-----------+----+------+---------+
|John|    Science|null|     F|     null|
|Ania|Mathematics|null|  null|       70|
+----+-----------+----+------+---------+



In [19]:
df3 = (df.filter(df.Mark.isNotNull()))
df3.show()

+------+-----------+----+------+---------+
|  name|    Subject|Mark|Status|Attedance|
+------+-----------+----+------+---------+
|Joseph|    Science|  86|     p|       90|
|Martin|Mathematics|  90|     p|     null|
| David|    English|  20|     F|       80|
| Diana|       null|  30|     F|       50|
+------+-----------+----+------+---------+



In [20]:
df4 = (df.filter((df.Mark.isNotNull()) & (df.Attedance.isNotNull()))) # bu ikili filtreleme mark ve attedance
# boş olmucak 2 side full olması lazım
df4.show()

+------+-------+----+------+---------+
|  name|Subject|Mark|Status|Attedance|
+------+-------+----+------+---------+
|Joseph|Science|  86|     p|       90|
| David|English|  20|     F|       80|
| Diana|   null|  30|     F|       50|
+------+-------+----+------+---------+



In [21]:
df4 = (df.filter((df.Mark.isNotNull()) | (df.Attedance.isNotNull()))) #  bunun anlamı şu ikiside null olursa
# getirme fakat biri bile dolu olursa getir sadece mark ve attedance için geçerli
df4.show()

+------+-----------+----+------+---------+
|  name|    Subject|Mark|Status|Attedance|
+------+-----------+----+------+---------+
|Joseph|    Science|  86|     p|       90|
|Martin|Mathematics|  90|     p|     null|
| David|    English|  20|     F|       80|
| Diana|       null|  30|     F|       50|
|  Ania|Mathematics|null|  null|       70|
+------+-----------+----+------+---------+



In [22]:
df5 = (df.na.fill(value=5))
df5.show()

+------+-----------+----+------+---------+
|  name|    Subject|Mark|Status|Attedance|
+------+-----------+----+------+---------+
|Joseph|    Science|  86|     p|       90|
|Martin|Mathematics|  90|     p|        5|
| David|    English|  20|     F|       80|
|  John|    Science|   5|     F|        5|
| Diana|       null|  30|     F|       50|
|  Ania|Mathematics|   5|  null|       70|
+------+-----------+----+------+---------+



In [23]:
df6 = (df.na.fill(value="NA"))
df6.show()

+------+-----------+----+------+---------+
|  name|    Subject|Mark|Status|Attedance|
+------+-----------+----+------+---------+
|Joseph|    Science|  86|     p|       90|
|Martin|Mathematics|  90|     p|     null|
| David|    English|  20|     F|       80|
|  John|    Science|null|     F|     null|
| Diana|         NA|  30|     F|       50|
|  Ania|Mathematics|null|    NA|       70|
+------+-----------+----+------+---------+



In [24]:
df5 = (df5.na.fill(value="NA"))
df5.show()

+------+-----------+----+------+---------+
|  name|    Subject|Mark|Status|Attedance|
+------+-----------+----+------+---------+
|Joseph|    Science|  86|     p|       90|
|Martin|Mathematics|  90|     p|        5|
| David|    English|  20|     F|       80|
|  John|    Science|   5|     F|        5|
| Diana|         NA|  30|     F|       50|
|  Ania|Mathematics|   5|    NA|       70|
+------+-----------+----+------+---------+



In [25]:
df7 = (df.na.fill({"Mark" : 0, "Status": "NA", "Name": "No_Name", "Subject": "English", "Attedance": 50}))
df7.show()

+------+-----------+----+------+---------+
|  name|    Subject|Mark|Status|Attedance|
+------+-----------+----+------+---------+
|Joseph|    Science|  86|     p|       90|
|Martin|Mathematics|  90|     p|       50|
| David|    English|  20|     F|       80|
|  John|    Science|   0|     F|       50|
| Diana|    English|  30|     F|       50|
|  Ania|Mathematics|   0|    NA|       70|
+------+-----------+----+------+---------+

